In [204]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [205]:
WELLCOME = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1") # The index is Date.  Add ,index_col=1 for making the date an index.
WELLCOME['COST'] = WELLCOME['COST (£) charged to Wellcome (inc VAT when charged)'] # create the "Cost" column without all the extraneous characters
WELLCOME['COST2'] = WELLCOME['COST'].apply(lambda x: x[1:]) # strip off the pound sterling sign
WELLCOME['COST2']=WELLCOME.COST2.apply(lambda x: ''.join(list(filter(str.isdigit, str(x))))).astype(float)/100 # return numeric
WELLCOME['Journal title']=WELLCOME['Journal title'].astype(str)  # set it to a string
WELLCOME['Article title']=WELLCOME['Article title'].astype(str)  # set it to a string
WELLCOME['Journal title2']=WELLCOME['Journal title'].str.lower() # set it to lower case so that we can group by the same case

In [152]:
WELLCOME.columns

Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)', 'COST', 'COST2'],
      dtype='object')

In [216]:
WELLCOME.dtypes

PMID/PMCID                                              object
Publisher                                               object
Journal title                                           object
Article title                                           object
COST (£) charged to Wellcome (inc VAT when charged)     object
COST                                                    object
COST2                                                  float64
Journal title2                                          object
dtype: object

In [217]:
WELLCOME_group_by_journal0 = WELLCOME[['Journal title2']].groupby(['Journal title2'])['Journal title2'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)

### Top 5 journals, with article counts

In [213]:
# Top 5 journals, with article counts
WELLCOME_group_by_journal0.head(5)

,Journal title2,count
773,plos one,190
510,journal of biological chemistry,53
701,neuroimage,29
767,plos genetics,24
774,plos pathogens,24


### Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal 
Note:  There are only standard deviation calculations for journal titles with more than 1 article.

In [214]:
# Grouping by region groupings
grouping_columns = ['Journal title','COST2']
sort_by_aggregate = ['mean', 'median','count' ,'std']

WELLCOME_group_by_journal = WELLCOME.groupby(grouping_columns[0])[grouping_columns].agg(sort_by_aggregate)
WELLCOME_group_by_journal = WELLCOME_group_by_journal_article.sort_values(by=['Journal title'], ascending=True)

In [215]:
WELLCOME_group_by_journal.head(20)
# WELLCOME.head(20)

COST2            \
                                                           mean    median   
Journal title                                                               
ACS Chemical Biology                                1418.186000  1294.590   
ACS Chemical Neuroscience                           1186.800000  1186.800   
ACS NANO                                             642.890000   642.890   
ACS Nano                                             693.390000   693.390   
ACTA F                                               754.900000   754.900   
AGE                                                 2002.000000  2002.000   
AIDS                                                2059.306667  1968.630   
AIDS Behav                                          1834.770000  1834.770   
AIDS Care                                           2189.170000  2189.170   
AIDS Journal                                        2015.720000  2015.720   
AIDS Research and Therapy                           1240.000000  1240.000   
AIDS UK                                             1836.920000  1836.920   
ASN Neuro                                           1453.420000  1453.420   
Academy of Nutrition and Dietetics                  2379.540000  2379.540   
Acta Crystallographica Section D,  Biological C...   771.420000   771.420   
Acta Crystallographica Section D: Biological Cr...   773.740000   773.740   
Acta Crystallographica Section F: Structural Bi...   796.635000   796.635   
Acta Crystallographica, Section D                    757.180000   757.180   
Acta Crystallography D                               774.190000   774.190   
Acta D                                               750.160000   750.160   

                                                                      
                                                   count         std  
Journal title                                                         
ACS Chemical Biology                                   5  507.309560  
ACS Chemical Neuroscience                              1         NaN  
ACS NANO                                               1         NaN  
ACS Nano                                               1         NaN  
ACTA F                                                 1         NaN  
AGE                                                    1         NaN  
AIDS                                                   3  281.067979  
AIDS Behav                                             1         NaN  
AIDS Care                                              2   61.617285  
AIDS Journal                                           1         NaN  
AIDS Research and Therapy                              1         NaN  
AIDS UK                                                2    0.000000  
ASN Neuro                                              1         NaN  
Academy of Nutrition and Dietetics                     1         NaN  
Acta Crystallographica Section D,  Biological C...     1         NaN  
Acta Crystallographica Section D: Biological Cr...     1         NaN  
Acta Crystallographica Section F: Structural Bi...     2   15.605847  
Acta Crystallographica, Section D                      1         NaN  
Acta Crystallography D                                 1         NaN  
Acta D                                                 1         NaN